In [19]:
%matplotlib inline

In [20]:
from global_settings import *


Chatbot Tutorial
================
**Author:** `Matthew Inkawhich <https://github.com/MatthewInkawhich>`_



In [21]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

Load & Preprocess Data
----------------------


Load and trim data
~~~~~~~~~~~~~~~~~~

Our next order of business is to create a vocabulary and load
query/response sentence pairs into memory.

Note that we are dealing with sequences of **words**, which do not have
an implicit mapping to a discrete numerical space. Thus, we must create
one by mapping each unique word that we encounter in our dataset to an
index value.

For this we define a ``Voc`` class, which keeps a mapping from words to
indexes, a reverse mapping of indexes to words, a count of each word and
a total word count. The class provides methods for adding a word to the
vocabulary (``addWord``), adding all words in a sentence
(``addSentence``) and trimming infrequently seen words (``trim``). More
on trimming later.




### Data preparation for translation task

In [50]:
import itertools

import torch

PAD_token = "<PAD>"
SOS_token = "<SOS>"
EOS_token = "<EOS>"
UNK_token = "<UNK>"

PAD_idx = 0
SOS_idx = 1
EOS_idx = 2
UNK_idx = 3

#### Mapping token - indixes #####

class Vocab:
    def __init__(self, lang):
        self.lang = lang
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_idx: PAD_token, SOS_idx: SOS_token, EOS_idx: EOS_token, UNK_idx: UNK_token}
        self.num_words = len(self.index2word.keys()) # Count default tokens

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_token(word)

    def add_token(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_idx: PAD_token, SOS_idx: SOS_token, EOS_idx: EOS_token, UNK_idx: UNK_token}

        self.num_words = len(self.index2word.keys()) # Count default tokens

        for word in keep_words:
            self.add_token(word)

    def __repr__(self):
        return "%s with %s unique tokens" %(self.lang, len(self.index2word.keys()))


    @staticmethod
    def build_vocab_from_pairs(sentence_list, lang):
        voc = Vocab(lang=lang)
        for sent in sentence_list:
            if isinstance(sent, list):
                print("Sentence is passed as list...", sent)
                sentence = ' '.join([word for word in sent])
                voc.add_sentence(sentence)
            else:
                voc.add_sentence(sent)
        return voc


#### Vectorization methods #####


def indexesFromSentence(voc, sentence):
    return [voc.word2index.get(word, 3) for word in sentence.split(' ')] + [EOS_idx]


def zeroPadding(l, fillvalue=PAD_idx):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

#TO use in combination with CrossEntropyLoss and ignore_index=0
def seq2paddedTensor(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    max_len = max([len(indexes) for indexes in indexes_batch])
    return padVar, lengths, max_len


# Returns all items for a given batch of pairs
def batch2TrainData(src_voc, tar_voc, pair_batch):
    #print(pair_batch[0])
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths, _ = seq2paddedTensor(input_batch, src_voc)
    output, out_lengths, max_tar_len = seq2paddedTensor(output_batch, tar_voc)
    return inp, lengths, output, out_lengths, max_tar_len

### Preprocessing

In [23]:
import io
import os
import string
import unicodedata
import re
from pickle import dump, load

from mappings import UMLAUT_MAP, ENG_CONTRACTIONS_MAP
#from tokenize import SOS_token, EOS_token
from global_settings import PREPRO_DIR

SOS_token = "<SOS>"
EOS_token = "<EOS>"


def read_lines(root, filename):
    path = os.path.join(root, filename)
    with io.open(path, encoding="utf-8", closefd=True) as f:
        lines = f.readlines()
    lines = [line.replace("\n", "").lower().split("\t") for line in lines]
    return lines

def reverse_language_pair(pairs):
    return [list(reversed(p)) for p in pairs]


def unicode_to_ascii(s):
    """
    Convert the string s to ascii while removing accents
    'Mn' = mark non-spacing automatically removes accents, e.g. Umlaute
    This is a more rapid cleaning, the meaning of the sentence can be compromised
    (e.g. in German this removes the conditional form of werden and converts it to
    the past simple 'wurde(n)'
    :param s:
    :return: cleaned s

    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def preprocess_sentence(sentence, expand_contractions=None, append_token=False):
    #print("Preprocessing sentence...")
    """
    Rapid preprocessing
    https://machinelearningmastery.com/prepare-french-english-dataset-machine-translation/
    Pipeline:
    - Expand contractions if needed
    - Apply some filters, e.g. filter out sentences starting with certain prefixes
    - Remove non printable chars
    - Remove punctuation
    - Normalize chars to latin chars (e.g. accents simply removed)
    - Splitting on white spaces (== tokenizing)
    - Lower case
    - Remove digits
    :param lines: lines to be preprocessed
    :return: cleaned lines
    """
    if expand_contractions:
        mapping = expand_contractions
        sentence = expand_contraction(sentence, mapping)

    #Filtering function only applies on a sentence level
    #if filter_func:
        #sentence = filter_func(sentence)

    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)

    line = unicodedata.normalize("NFD", sentence).encode("ascii", "ignore")
    line = line.decode('UTF-8')
    # tokenize on white space
    line = line.strip().split(" ")
    # convert to lower case
    line = [word.lower() for word in line]
    # remove punctuation from each token
    line = [word.translate(table) for word in line]
    # remove non-printable chars form each token
    line = [re_print.sub('', w) for w in line]
    # remove tokens with numbers in them
    line = [word for word in line if word.isalpha()]

    line = ' '.join([word for word in line])

   # print(line)

    if append_token:
        line = [SOS_token] + line + [EOS_token]
    return line



def expand_contraction(sentence, mapping):
    """
    Expands tokens in sentence given a contraction dictionary

    source: https://www.linkedin.com/pulse/processing-normalizing-text-data-saurav-ghosh

    :param sentence: sentence to expand
    :param mapping: contraction dictionary
    :return: expanded sentence
    """
    contractions_patterns = re.compile('({})'.format('|'.join(mapping.keys())), flags=re.IGNORECASE | re.DOTALL)

    def replace_text(t):
        txt = t.group(0)
        if txt.lower() in mapping.keys():
            return mapping[txt.lower()]

    expanded_sentence = contractions_patterns.sub(replace_text, sentence)
    return expanded_sentence


def normalize_string(sentence):
    """
    Inspired by: PyTorch Tutorials
    Used in combination to @unicode_to_ascii function
    :param sentence:
    :param replace_dict:
    :return:
    """
    s = sentence
    s = re.sub(r"([ß])", r"ss", s)  # unicode_to_ascii cannot handle ß

    s = unicode_to_ascii(s)
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z0-9.!?]+", r" ", s)

    return s


def reverse_order(token_list, reverse=True):
    """
    :param token_list:
    :param reverse: Flag
    :return: reversed token list or token list if filter is not to apply
    """
    if reverse:
        token_list = token_list[::-1]
    return token_list


def save_clean_data(path, pairs, filename):
    path_to_dir = os.path.join(path, filename)
    dump(pairs, open(path_to_dir, 'wb'))
    print('File %s saved in %s' % (filename, path_to_dir))

def load_cleaned_data(path, filename):
    path_to_file = os.path.join(path, filename)
    if(os.path.isfile(path_to_file)):
        return load(open(path_to_file, 'rb'))
    else: raise RuntimeError("File not found, please preprocess and save sentences!")


def preprocess_pipeline(pairs, cleaned_file_to_store=None, exp_contraction=None, reverse_pairs=False):
    """
    Assuming english as input language
    :param cleaned_file_to_store:
    :param exp_contraction:
    :param reverse_pairs:
    :return:
    """

   # pairs = read_lines(DATA_DIR, FILENAME)
  #  print(len(pairs))
    #print(pairs[10])
    src_sents = [item[0] for item in pairs]
    trg_sents = [item[1] for item in pairs]
    if expand_contraction:
        src_mapping = ENG_CONTRACTIONS_MAP
        trg_mapping = UMLAUT_MAP
    else:
        src_mapping = exp_contraction
        trg_mapping = exp_contraction

    cleaned_pairs = []
    for i, (src_sent, trg_sent) in enumerate(pairs):
        cleaned_src_sent = preprocess_sentence(src_sent, src_mapping)
        cleaned_trg_sent = preprocess_sentence(trg_sent, trg_mapping)
        cleaned_list = [cleaned_src_sent, cleaned_trg_sent]

        cleaned_pairs.append(cleaned_list)

    if reverse_pairs:
        cleaned_pairs = reverse_language_pair(cleaned_pairs)

    if cleaned_file_to_store:
        save_clean_data(PREPRO_DIR, cleaned_pairs, cleaned_file_to_store)
    return cleaned_pairs

In [24]:
from global_settings import DATA_DIR
src_lang = "eng"
trg_lang = "deu"
exp_contraction = True
src_reversed = False
limit = None

start_root = "."
pairs = read_lines(os.path.join(start_root, DATA_DIR), FILENAME)
print(len(pairs))
print(pairs[10])

cleaned_file = "%s-%s_cleaned" % (src_lang, trg_lang) + "_rude" if not exp_contraction else "%s-%s_cleaned" % (
src_lang, trg_lang) + "_full"
cleaned_file = cleaned_file + "reversed.pkl" if src_reversed else cleaned_file + ".pkl"

pairs = preprocess_pipeline(pairs, cleaned_file, exp_contraction)

if limit:
    pairs = pairs[:limit]


176692
['go on.', 'mach weiter.']
File eng-deu_cleaned_full.pkl saved in data/prepro/eng-deu_cleaned_full.pkl


In [26]:
src_sents = [item[0] for item in pairs]
trg_sents = [item[1] for item in pairs]

In [30]:
def build_vocab(sent_list, lang_name):
    vocab = Voc(name=lang_name)
    for sent in sent_list:
        vocab.addSentence(sent)
    return vocab

In [37]:
input_lang = build_vocab(src_sents, "eng")
output_lang = build_vocab(trg_sents, "deu")

Now we can assemble our vocabulary and query/response sentence pairs.
Before we are ready to use this data, we must perform some
preprocessing.

First, we must convert the Unicode strings to ASCII using
``unicodeToAscii``. Next, we should convert all letters to lowercase and
trim all non-letter characters except for basic punctuation
(``normalizeString``). Finally, to aid in training convergence, we will
filter out sentences with length greater than the ``MAX_LENGTH``
threshold (``filterPairs``).




Another tactic that is beneficial to achieving faster convergence during
training is trimming rarely used words out of our vocabulary. Decreasing
the feature space will also soften the difficulty of the function that
the model must learn to approximate. We will do this as a two-step
process:

1) Trim words used under ``MIN_COUNT`` threshold using the ``voc.trim``
   function.

2) Filter out pairs with trimmed words.




In [38]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


#Trim voc and pairs
#pairs = trimRareWords(input_lang, pairs, MIN_COUNT)

Prepare Data for Models
-----------------------

Although we have put a great deal of effort into preparing and massaging our
data into a nice vocabulary object and list of sentence pairs, our models
will ultimately expect numerical torch tensors as inputs. One way to
prepare the processed data for the models can be found in the `seq2seq
translation
tutorial <https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html>`__.
In that tutorial, we use a batch size of 1, meaning that all we have to
do is convert the words in our sentence pairs to their corresponding
indexes from the vocabulary and feed this to the models.

However, if you’re interested in speeding up training and/or would like
to leverage GPU parallelization capabilities, you will need to train
with mini-batches.

Using mini-batches also means that we must be mindful of the variation
of sentence length in our batches. To accomodate sentences of different
sizes in the same batch, we will make our batched input tensor of shape
*(max_length, batch_size)*, where sentences shorter than the
*max_length* are zero padded after an *EOS_token*.

If we simply convert our English sentences to tensors by converting
words to their indexes(\ ``indexesFromSentence``) and zero-pad, our
tensor would have shape *(batch_size, max_length)* and indexing the
first dimension would return a full sequence across all time-steps.
However, we need to be able to index our batch along time, and across
all sequences in the batch. Therefore, we transpose our input batch
shape to *(max_length, batch_size)*, so that indexing across the first
dimension returns a time step across all sentences in the batch. We
handle this transpose implicitly in the ``zeroPadding`` function.

.. figure:: /_static/img/chatbot/seq2seq_batches.png
   :align: center
   :alt: batches

The ``inputVar`` function handles the process of converting sentences to
tensor, ultimately creating a correctly shaped zero-padded tensor. It
also returns a tensor of ``lengths`` for each of the sequences in the
batch which will be passed to our decoder later.

The ``outputVar`` function performs a similar function to ``inputVar``,
but instead of returning a ``lengths`` tensor, it returns a binary mask
tensor and a maximum target sentence length. The binary mask tensor has
the same shape as the output target tensor, but every element that is a
*PAD_token* is 0 and all others are 1.

``batch2TrainData`` simply takes a bunch of pairs and returns the input
and target tensors using the aforementioned functions.




In [39]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index.get(word, 3) for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(src_voc, tar_voc, pair_batch):
    print(type(pair_batch))
    print(pair_batch[0])
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, src_voc)
    output, mask, max_target_len = outputVar(output_batch, tar_voc)
    return inp, lengths, output, mask, max_target_len



In [34]:
pairs[20]

['freeze', 'keine bewegung']

In [35]:
print(pairs[0][0])
print(pairs[0][1])

hi
hallo


In [40]:
# Example for validation
small_batch_size = 5
batches = batch2TrainData(input_lang, output_lang, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches
print("Sentence 1:", pairs[0][1])
print("Test indexesFromSentence:", indexesFromSentence(output_lang, pairs[0][1]))
print("Sentence 2:", pairs[0][0])
print("Test indexesFromSentence:", indexesFromSentence(input_lang, pairs[0][0]))

#print(inputVar(pairs[1][0], voc))
#print(" ")
print("input_variable:\n", input_variable)
print("lengths:\n", lengths)
print("target_variable:\n", target_variable)
print("mask:\n", mask)
print("max_target_len:\n", max_target_len)

<class 'list'>
['my elbow still hurts', 'mein ellbogen tut immer noch weh']
Sentence 1: hallo
Test indexesFromSentence: [4, 2]
Sentence 2: hi
Test indexesFromSentence: [4, 2]
input_variable:
 tensor([[  14,  426,  109,  424,  424],
        [  35,   72,  790,  866, 2066],
        [ 255,   38,  175,   72,  282],
        [ 205, 3740,  105,  255,  144],
        [  81,  178, 1256,  126,    2],
        [  44,   24,   79,    2,    0],
        [ 205,   72,  326,    0,    0],
        [2224, 2431,    2,    0,    0],
        [1186,    2,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths:
 tensor([10,  9,  8,  6,  5])
target_variable:
 tensor([[  18, 1382,  233,  986,  986],
        [1353,  239, 2892, 1538, 3378],
        [  84,   69,  165, 3301,  283],
        [  12,  318,  136,  266, 1394],
        [1035,  231,  221,  267,  604],
        [   2,   36,  452,    2,  284],
        [   0,  752, 2279,    0,    2],
        [   0, 2902,    2,    0,    0],
        [   0,    2,    0,    

Define Models
-------------

Seq2Seq Model
~~~~~~~~~~~~~

The brains of our chatbot is a sequence-to-sequence (seq2seq) model. The
goal of a seq2seq model is to take a variable-length sequence as an
input, and return a variable-length sequence as an output using a
fixed-sized model.

`Sutskever et al. <https://arxiv.org/abs/1409.3215>`__ discovered that
by using two separate recurrent neural nets together, we can accomplish
this task. One RNN acts as an **encoder**, which encodes a variable
length input sequence to a fixed-length context vector. In theory, this
context vector (the final hidden layer of the RNN) will contain semantic
information about the query sentence that is input to the bot. The
second RNN is a **decoder**, which takes an input word and the context
vector, and returns a guess for the next word in the sequence and a
hidden state to use in the next iteration.

.. figure:: /_static/img/chatbot/seq2seq_ts.png
   :align: center
   :alt: model

Image source:
https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_intro/




Encoder
~~~~~~~

The encoder RNN iterates through the input sentence one token
(e.g. word) at a time, at each time step outputting an “output” vector
and a “hidden state” vector. The hidden state vector is then passed to
the next time step, while the output vector is recorded. The encoder
transforms the context it saw at each point in the sequence into a set
of points in a high-dimensional space, which the decoder will use to
generate a meaningful output for the given task.

At the heart of our encoder is a multi-layered Gated Recurrent Unit,
invented by `Cho et al. <https://arxiv.org/pdf/1406.1078v3.pdf>`__ in
2014. We will use a bidirectional variant of the GRU, meaning that there
are essentially two independent RNNs: one that is fed the input sequence
in normal sequential order, and one that is fed the input sequence in
reverse order. The outputs of each network are summed at each time step.
Using a bidirectional GRU will give us the advantage of encoding both
past and future context.

Bidirectional RNN:

.. figure:: /_static/img/chatbot/RNN-bidirectional.png
   :width: 70%
   :align: center
   :alt: rnn_bidir

Image source: https://colah.github.io/posts/2015-09-NN-Types-FP/

Note that an ``embedding`` layer is used to encode our word indices in
an arbitrarily sized feature space. For our models, this layer will map
each word to a feature space of size *hidden_size*. When trained, these
values should encode semantic similarity between similar meaning words.

Finally, if passing a padded batch of sequences to an RNN module, we
must pack and unpack padding around the RNN pass using
``torch.nn.utils.rnn.pack_padded_sequence`` and
``torch.nn.utils.rnn.pad_packed_sequence`` respectively.

**Computation Graph:**

   1) Convert word indexes to embeddings.
   2) Pack padded batch of sequences for RNN module.
   3) Forward pass through GRU.
   4) Unpack padding.
   5) Sum bidirectional GRU outputs.
   6) Return output and final hidden state.

**Inputs:**

-  ``input_seq``: batch of input sentences; shape=\ *(max_length,
   batch_size)*
-  ``input_lengths``: list of sentence lengths corresponding to each
   sentence in the batch; shape=\ *(batch_size)*
-  ``hidden``: hidden state; shape=\ *(n_layers x num_directions,
   batch_size, hidden_size)*

**Outputs:**

-  ``outputs``: output features from the last hidden layer of the GRU
   (sum of bidirectional outputs); shape=\ *(max_length, batch_size,
   hidden_size)*
-  ``hidden``: updated hidden state from GRU; shape=\ *(n_layers x
   num_directions, batch_size, hidden_size)*





In [41]:
class EncoderLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, rnn_hidden_size, n_layers=1, dropout = 0, bidirectional=False):
        super(EncoderLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.rnn_hidden_size = rnn_hidden_size
        self.n_layers = n_layers
        self.bidirectional = bidirectional

        #Layers
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(vocab_size, emb_dim,padding_idx=0)
        self.rnn = nn.LSTM(emb_dim, rnn_hidden_size, num_layers=n_layers, bidirectional=bidirectional)

    def forward(self, input_seq, input_lengths=None):

        seq_embedded = self.dropout(self.embedding(input_seq))

        if not input_lengths is None:
            seq_embedded = torch.nn.utils.rnn.pack_padded_sequence(seq_embedded, input_lengths)

            enc_outputs, (enc_hidden, enc_cell) = self.rnn(seq_embedded)

            enc_outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(enc_outputs)
        else:
            enc_outputs, (enc_hidden, enc_cell) = self.rnn(seq_embedded)

        return enc_outputs, enc_hidden, enc_cell

In [42]:
class DecoderLSTM(nn.Module):
    def __init__(self, vocab_size, emb_dim, rnn_hidden_size, sos_idx=1, n_layers=1, dropout=0):
        super(DecoderLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.rnn_hidden_size = rnn_hidden_size
        self.sos_idx = sos_idx
        self.n_layers = n_layers

        # Layers
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(emb_dim, rnn_hidden_size, num_layers=n_layers, dropout=dropout)
        self.out = nn.Linear(rnn_hidden_size, vocab_size)


    def init_input(self, batch_size):
        #Initializes first vector with only SOS idx (1)
        #torch.LongTensor([[1 for _ in range(small_batch_size)]])
        return torch.LongTensor([[self.sos_idx for _ in range(batch_size)]]).to(device)

        #input_step, last_hidden, encoder_outputs
    def forward(self, input_step, last_hidden, context):
        #input_step shape [1,batch_size]
        embedded = self.dropout(self.embedding(input_step))

        output, (hidden, cell) = self.rnn(embedded, (last_hidden, context))

        output= output.squeeze(0)

        output = self.out(output)

        return output, hidden, cell

In [45]:
class NMTModel(nn.Module):
    def __init__(self, encoder:EncoderLSTM, decoder:DecoderLSTM, device=device):
        super(NMTModel, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        self.encoder = self.encoder.to(device)
        self.decoder = self.decoder.to(device)

    def forward(self, src_input, trg_input, teacher_forcing_ratio=0.3, src_lengths=None):

        #Sequences are stored as (seq_len, batch_size)
        #Batch size is the same for both input and target
        #seq_len can differ. As the sequence in the decoding phase is processed one timestep at time
        #seq_len is set to the max length in the target batch

        batch_size = trg_input.shape[1]
        seq_len = trg_input.shape[0]

        output_size = self.decoder.vocab_size

        dec_outputs = torch.zeros(seq_len, batch_size, output_size).to(device)

        src_input = src_input.to(device)

        #Encode the input sequences

        output, hidden, cell = self.encoder(src_input, src_lengths)

        decoder_input = self.decoder.init_input(batch_size) #SOS
        #hidden = hidden[:self.decoder.n_layers]

        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False


        #Run through target row by row
        for timestep in range(seq_len):
            output, hidden, cell = self.decoder(decoder_input, hidden, cell)
            #store results
            dec_outputs[timestep] = output

            """
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
            """

            if use_teacher_forcing:
                # determine teacher forced input
                decoder_input = trg_input[timestep].view(1,-1) #real target value
            else:
                #Use prediction
                _, topi = output.topk(1)
                decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
                decoder_input = decoder_input.to(device)
        return dec_outputs

Now that we have defined our attention submodule, we can implement the
actual decoder model. For the decoder, we will manually feed our batch
one time step at a time. This means that our embedded word tensor and
GRU output will both have shape *(1, batch_size, hidden_size)*.

**Computation Graph:**

   1) Get embedding of current input word.
   2) Forward through unidirectional GRU.
   3) Calculate attention weights from the current GRU output from (2).
   4) Multiply attention weights to encoder outputs to get new "weighted sum" context vector.
   5) Concatenate weighted context vector and GRU output using Luong eq. 5.
   6) Predict next word using Luong eq. 6 (without softmax).
   7) Return output and final hidden state.

**Inputs:**

-  ``input_step``: one time step (one word) of input sequence batch;
   shape=\ *(1, batch_size)*
-  ``last_hidden``: final hidden layer of GRU; shape=\ *(n_layers x
   num_directions, batch_size, hidden_size)*
-  ``encoder_outputs``: encoder model’s output; shape=\ *(max_length,
   batch_size, hidden_size)*

**Outputs:**

-  ``output``: softmax normalized tensor giving probabilities of each
   word being the correct next word in the decoded sequence;
   shape=\ *(batch_size, voc.num_words)*
-  ``hidden``: final hidden state of GRU; shape=\ *(n_layers x
   num_directions, batch_size, hidden_size)*




Define Training Procedure
-------------------------

Masked loss
~~~~~~~~~~~

Since we are dealing with batches of padded sequences, we cannot simply
consider all elements of the tensor when calculating loss. We define
``maskNLLLoss`` to calculate our loss based on our decoder’s output
tensor, the target tensor, and a binary mask tensor describing the
padding of the target tensor. This loss function calculates the average
negative log likelihood of the elements that correspond to a *1* in the
mask tensor.




In [43]:
def maskNLLLoss(inp, target, mask):
    #print(inp.shape)
   # print(target.shape)
    #print(mask.shape)
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Single training iteration
~~~~~~~~~~~~~~~~~~~~~~~~~

The ``train`` function contains the algorithm for a single training
iteration (a single batch of inputs).

We will use a couple of clever tricks to aid in convergence:

-  The first trick is using **teacher forcing**. This means that at some
   probability, set by ``teacher_forcing_ratio``, we use the current
   target word as the decoder’s next input rather than using the
   decoder’s current guess. This technique acts as training wheels for
   the decoder, aiding in more efficient training. However, teacher
   forcing can lead to model instability during inference, as the
   decoder may not have a sufficient chance to truly craft its own
   output sequences during training. Thus, we must be mindful of how we
   are setting the ``teacher_forcing_ratio``, and not be fooled by fast
   convergence.

-  The second trick that we implement is **gradient clipping**. This is
   a commonly used technique for countering the “exploding gradient”
   problem. In essence, by clipping or thresholding gradients to a
   maximum value, we prevent the gradients from growing exponentially
   and either overflow (NaN), or overshoot steep cliffs in the cost
   function.

.. figure:: /_static/img/chatbot/grad_clip.png
   :align: center
   :width: 60%
   :alt: grad_clip

Image source: Goodfellow et al. *Deep Learning*. 2016. https://www.deeplearningbook.org/

**Sequence of Operations:**

   1) Forward pass entire input batch through encoder.
   2) Initialize decoder inputs as SOS_token, and hidden state as the encoder's final hidden state.
   3) Forward input batch sequence through decoder one time step at a time.
   4) If teacher forcing: set next decoder input as the current target; else: set next decoder input as current decoder output.
   5) Calculate and accumulate loss.
   6) Perform backpropagation.
   7) Clip gradients.
   8) Update encoder and decoder model parameters.


.. Note ::

  PyTorch’s RNN modules (``RNN``, ``LSTM``, ``GRU``) can be used like any
  other non-recurrent layers by simply passing them the entire input
  sequence (or batch of sequences). We use the ``GRU`` layer like this in
  the ``encoder``. The reality is that under the hood, there is an
  iterative process looping over each time step calculating hidden states.
  Alternatively, you ran run these modules one time-step at a time. In
  this case, we manually loop over the sequences during the training
  process like we must do for the ``decoder`` model. As long as you
  maintain the correct conceptual model of these modules, implementing
  sequential models can be very straightforward.





In [63]:
def train(input_variable, lengths, target_variable, mask, 
          max_target_len, encoder, decoder, src_embedding, 
          trg_embedding,
          encoder_optimizer, 
          decoder_optimizer, batch_size, 
          clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    rand = random.random()
    #print(rand)
    use_teacher_forcing = True if rand < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Training iterations
~~~~~~~~~~~~~~~~~~~

It is finally time to tie the full training procedure together with the
data. The ``trainIters`` function is responsible for running
``n_iterations`` of training given the passed models, optimizers, data,
etc. This function is quite self explanatory, as we have done the heavy
lifting with the ``train`` function.

One thing to note is that when we save our model, we save a tarball
containing the encoder and decoder state_dicts (parameters), the
optimizers’ state_dicts, the loss, the iteration, etc. Saving the model
in this way will give us the ultimate flexibility with the checkpoint.
After loading a checkpoint, we will be able to use the model parameters
to run inference, or we can continue training right where we left off.




In [64]:
def trainIters(model_name, src_voc, tar_voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, src_embedding, trg_embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,print_every, save_every, clip, corpus_name, loadFilename):
    # Load batches for each iteration
    training_batches = [batch2TrainData(src_voc, tar_voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, src_emb, trg_emb, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'src_dict': src_voc.__dict__,
                'tar_dict': tar_voc.__dict__,
                'src_embedding': src_embedding.state_dict(),
                'trg_embedding': trg_embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

Define Evaluation
-----------------

After training a model, we want to be able to talk to the bot ourselves.
First, we must define how we want the model to decode the encoded input.

Greedy decoding
~~~~~~~~~~~~~~~

Greedy decoding is the decoding method that we use during training when
we are **NOT** using teacher forcing. In other words, for each time
step, we simply choose the word from ``decoder_output`` with the highest
softmax value. This decoding method is optimal on a single time-step
level.

To facilite the greedy decoding operation, we define a
``GreedySearchDecoder`` class. When run, an object of this class takes
an input sequence (``input_seq``) of shape *(input_seq length, 1)*, a
scalar input length (``input_length``) tensor, and a ``max_length`` to
bound the response sentence length. The input sentence is evaluated
using the following computational graph:

**Computation Graph:**

   1) Forward input through encoder model.
   2) Prepare encoder's final hidden layer to be first hidden input to the decoder.
   3) Initialize decoder's first input as SOS_token.
   4) Initialize tensors to append decoded words to.
   5) Iteratively decode one word token at a time:
       a) Forward pass through decoder.
       b) Obtain most likely word token and its softmax score.
       c) Record token and score.
       d) Prepare current token to be next decoder input.
   6) Return collections of word tokens and scores.




In [49]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

Evaluate my text
~~~~~~~~~~~~~~~~

Now that we have our decoding method defined, we can write functions for
evaluating a string input sentence. The ``evaluate`` function manages
the low-level process of handling the input sentence. We first format
the sentence as an input batch of word indexes with *batch_size==1*. We
do this by converting the words of the sentence to their corresponding
indexes, and transposing the dimensions to prepare the tensor for our
models. We also create a ``lengths`` tensor which contains the length of
our input sentence. In this case, ``lengths`` is scalar because we are
only evaluating one sentence at a time (batch_size==1). Next, we obtain
the decoded response sentence tensor using our ``GreedySearchDecoder``
object (``searcher``). Finally, we convert the response’s indexes to
words and return the list of decoded words.

``evaluateInput`` acts as the user interface for our chatbot. When
called, an input text field will spawn in which we can enter our query
sentence. After typing our input sentence and pressing *Enter*, our text
is normalized in the same way as our training data, and is ultimately
fed to the ``evaluate`` function to obtain a decoded output sentence. We
loop this process, so we can keep chatting with our bot until we enter
either “q” or “quit”.

Finally, if a sentence is entered that contains a word that is not in
the vocabulary, we handle this gracefully by printing an error message
and prompting the user to enter another sentence.




In [65]:
def evaluate(encoder, decoder, searcher, src_voc, trg_voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(src_voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [trg_voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher,  src_voc, trg_voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, src_voc, trg_voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Translation:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

Run Model
---------

Finally, it is time to run our model!

Regardless of whether we want to train or test the chatbot model, we
must initialize the individual encoder and decoder models. In the
following block, we set our desired configurations, choose to start from
scratch or set a checkpoint to load from, and build and initialize the
models. Feel free to play with different model configurations to
optimize performance.




In [66]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
src_emb = nn.Embedding(input_lang.num_words, hidden_size)
trg_emb = nn.Embedding(output_lang.num_words, hidden_size)

if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, src_emb, encoder_n_layers, dropout)
#decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
decoder = VanillaDecoderRNN(trg_emb, hidden_size, output_lang.num_words, decoder_n_layers, dropout)

if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')
print(encoder)
print(decoder)

Building encoder and decoder ...
Models built and ready to go!
EncoderRNN(
  (embedding): Embedding(24307, 500)
  (gru): GRU(500, 500, num_layers=2, dropout=0.1)
)
VanillaDecoderRNN(
  (embedding): Embedding(12416, 500)
  (embedding_dropout): Dropout(p=0.1)
  (gru): GRU(500, 500, num_layers=2, dropout=0.1)
  (out): Linear(in_features=500, out_features=12416, bias=True)
)


Run Training
~~~~~~~~~~~~

Run the following block if you want to train the model.

First we set training parameters, then we initialize our optimizers, and
finally we call the ``trainIters`` function to run our training
iterations.




In [67]:
# Configure training/optimization
clip = 30.0
teacher_forcing_ratio = 0.3
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 10000
print_every = 100
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
trainIters(model_name, input_lang, output_lang, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           src_emb, trg_emb, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, "eng-deu.txt", loadFilename)


Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 100; Percent complete: 1.0%; Average loss: 5.4819
Iteration: 200; Percent complete: 2.0%; Average loss: 4.6225
Iteration: 300; Percent complete: 3.0%; Average loss: 4.4058
Iteration: 400; Percent complete: 4.0%; Average loss: 4.2229
Iteration: 500; Percent complete: 5.0%; Average loss: 4.1600
Iteration: 600; Percent complete: 6.0%; Average loss: 4.0188
Iteration: 700; Percent complete: 7.0%; Average loss: 3.9230
Iteration: 800; Percent complete: 8.0%; Average loss: 3.8660
Iteration: 900; Percent complete: 9.0%; Average loss: 3.7455
Iteration: 1000; Percent complete: 10.0%; Average loss: 3.6952
Iteration: 1100; Percent complete: 11.0%; Average loss: 3.5884
Iteration: 1200; Percent complete: 12.0%; Average loss: 3.5597
Iteration: 1300; Percent complete: 13.0%; Average loss: 3.5066
Iteration: 1400; Percent complete: 14.0%; Average loss: 3.4421
Iteration: 1500; Percent complete: 15.0%; Average loss: 3.4133
I

"""
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 8.9542
Iteration: 2; Percent complete: 0.1%; Average loss: 8.8097
Iteration: 3; Percent complete: 0.1%; Average loss: 8.6029
Iteration: 4; Percent complete: 0.1%; Average loss: 8.2978
Iteration: 5; Percent complete: 0.1%; Average loss: 7.9560
"""

Run Evaluation
~~~~~~~~~~~~~~

To chat with your model, run the following block.




In [68]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

In [69]:

# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, input_lang, output_lang)

> er ist dj
Translation: he s a dj .
> er ist freundlich
Translation: he s a .
> er ist liebenswürdig
Translation: he s not .
> er ist gemein
Translation: he is a .
> wie klug!
Translation: how clever !
> wie nett
Translation: how s a .
> ich bin ein junge
Translation: i m a boy .
> ich bin ein mädchen
Translation: i m a girl .
> ich gebe es zu
Translation: i m it to .
> ich will essen
Translation: i want to eat .
> ich will schlafen
Translation: i want to sleep .
> er will essen
Translation: he wants to eat .
> er will schlafen
Translation: he wants to sleep .
> sie wollen schlafen
Translation: they want to go .
> sie wollen reisen
Translation: they want to . .
> ich möchte reisen
Translation: i want to . . .
> ich will reisen
Translation: i want to . .
> ich will lesen
Translation: i want to read .
> er will lesen
Translation: he wants to read .
> wie tragisch
Translation: how s cats .
> er liebt Katzen
Translation: he loves cats .
> sie liebt Katzen
Translation: she loves cats .
> i

Conclusion
----------

That’s all for this one, folks. Congratulations, you now know the
fundamentals to building a generative chatbot model! If you’re
interested, you can try tailoring the chatbot’s behavior by tweaking the
model and training parameters and customizing the data that you train
the model on.

Check out the other tutorials for more cool deep learning applications
in PyTorch!


